# Welter issue #9
## Generate synthetic, noised-up two-temperature model spectra, then naively fit a single temperature model to it.
### Part 2- Prepare.

Michael Gully-Santiago  
Friday, January 8, 2015  

See the previous notebook for the theory and background.

Steps:
1. Modify all the config and phi files to have the values from the MCMC run.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_context('paper', font_scale=1.4)
sns.set_style('ticks')

In [3]:
import os
import json
import pandas as pd
import yaml
import h5py

In [4]:
sf_dat = pd.read_csv('../data/analysis/IGRINS_ESPaDOnS_run01_last10kMCMC.csv')
sf_dat.rename(columns={"m_val_x":"m_val"}, inplace=True)
del sf_dat['m_val_y']

## Set the value of the config files to all the same values.

In [5]:
ms = sf_dat.m_val

In [6]:
for m in ms:
    index = sf_dat.index[sf_dat.m_val == m]
    mdir = 'eo{:03d}'.format(m)
    sf_out = '../sf/eo{:03d}/config.yaml'.format(m)

    f2 = open(sf_out)
    config = yaml.load(f2)
    f2.close()

    ii = index.values[0]
    
    config['Theta']['grid'] = [4100.0, 3.5, 0.0]
    config['Theta']['vsini'] = float(28.5)
    config['Theta']['vz'] = float(15.6)
    config['Theta']['vz'] = float(15.6)
    config['Theta']['logOmega'] = float(-0.07)
    if (sf_dat.logO_50p[ii] == sf_dat.logO_50p[ii]):
        config['Theta']['logOmega'] = float(sf_dat.logO_50p[ii])

    with open(sf_out, mode='w') as outfile:
        outfile.write(yaml.dump(config))

In [7]:
for m in ms:
    index = sf_dat.index[sf_dat.m_val == m]
    mdir = 'eo{:03d}'.format(m)
    
    phi_out = '../sf/eo{:03d}/s0_o0phi.json'.format(m)

    jf = open(phi_out)
    phi = json.load(jf)
    jf.close()

    ii = index.values[0]
    
    c1, c2, c3 = sf_dat.c1_50p[ii], sf_dat.c2_50p[ii], sf_dat.c3_50p[ii]
    if c1 != c1:
        print("default: {}".format(m))
        phi['cheb'] = [0.0,0,0]
        phi['logAmp']= -1.6
        phi['sigAmp']= 1.0
        phi['l']= 30.0
    if c1 == c1:
        print("actual: {}".format(m))
        phi['cheb'] = [c1, c2, c3]
        if sf_dat.LA_50p[ii] > -1.4:
            phi['logAmp']= -1.4
        else:
            phi['logAmp']= sf_dat.LA_50p[ii]
        phi['sigAmp']= sf_dat.SA_50p[ii]
        phi['l']= sf_dat.ll_50p[ii]
    
    phi['fix_c0'] = True
    
    with open(phi_out, mode='w') as outfile:
        json.dump(phi, outfile, indent=2)

actual: 0
actual: 1
actual: 2
actual: 3
actual: 4
actual: 5
actual: 6
actual: 7
actual: 8
actual: 9
actual: 10
actual: 11
actual: 12
actual: 13
actual: 14
actual: 15
actual: 16
actual: 17
actual: 18
default: 19
actual: 20
actual: 21
actual: 22
actual: 23
default: 24
default: 25
actual: 26
default: 27
default: 28
actual: 29
actual: 30
default: 31
default: 32
default: 33
default: 34


The end.